# **A/B Testing İle Bidding Yöntemlerinin Karşılaştırılması**

**Problem**

Dünyaca ünlü bir şirket teklif verme türüne alternatif olarak yeni bir teklif türü tanıtmıştır. Bu yeni teklif türünün daha fazla dönüşüm getirip getirmediği test edilmek istenmektedir.

Bu noktada A/B Testi uygulanacaktır. Başarı ölçütü Purchase'dir. İstatistiksel testler için Purchase metriğine odaklanılacaktır.

**Veri Setini Tanıyalım**


Kontrol grubu ve Test grubu olmak üzere iki ayrı veri seti bulunmaktadır. Kontrol grubuna eski teklif türü, Test grubuna ise yeni teklif türü uygulanmıştır. Veri setinde ayrıca kullanıcıların gördükleri ve tıkladıkları reklam sayıları ve buradan gelen kazanç bilgileri yer almaktadır.

**Değişkenler**


Impression: Reklam görüntüleme sayısı

Click: Görüntülenen reklama tıklama sayısı

Purchase: Tıklanan reklamlar sonrası satın alınan ürün sayısı

Earning: Satın alınan ürünler sonrası elde edilen kazanç

**Veriyi Hazırlama**


Gerekli olan kütüphaneler indirilir.

In [1]:
import pandas as pd
df1=pd.read_excel("/kaggle/input/ab-testing-data/ab_testing.xlsx", sheet_name = "Control Group")
df2=pd.read_excel("/kaggle/input/ab-testing-data/ab_testing.xlsx", sheet_name = "Test Group")

In [2]:
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

In [3]:
#Kontrol Grubu Veri Seti

df1.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [4]:
#Test Grubu Veri Seti

df2.head()

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


**Kontrol ve Test Gruplarının Analizi**

In [5]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head())
    print("##################### Tail #####################")
    print(dataframe.tail())
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Descriptive Statistics #####################")
    print(dataframe.describe().T)
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.25, 0.50, 0.75, 0.95, 0.99, 1]).T)

In [6]:
#Kontrol Grubu Analizi

check_df(df1)

##################### Shape #####################
(40, 4)
##################### Types #####################
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
##################### Head #####################
      Impression        Click    Purchase      Earning
0   82529.459271  6090.077317  665.211255  2311.277143
1   98050.451926  3382.861786  315.084895  1742.806855
2   82696.023549  4167.965750  458.083738  1797.827447
3  109914.400398  4910.882240  487.090773  1696.229178
4  108457.762630  5987.655811  441.034050  1543.720179
##################### Tail #####################
       Impression        Click    Purchase      Earning
35  132064.219003  3747.157544  551.072406  2256.975589
36   86409.941796  4608.256205  345.046033  1781.357690
37  123678.934234  3649.073787  476.168128  2187.721217
38  101997.494099  4736.353369  474.613537  2254.563834
39  121085.881220  4285.178608  590.406020  1289.308948
##################### NA #

In [7]:
#Test Grubu Analizi

check_df(df2)

##################### Shape #####################
(40, 4)
##################### Types #####################
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
##################### Head #####################
      Impression        Click    Purchase      Earning
0  120103.503796  3216.547958  702.160346  1939.611243
1  134775.943363  3635.082422  834.054286  2929.405820
2  107806.620788  3057.143560  422.934258  2526.244877
3  116445.275526  4650.473911  429.033535  2281.428574
4  145082.516838  5201.387724  749.860442  2781.697521
##################### Tail #####################
       Impression        Click    Purchase      Earning
35   79234.911929  6002.213585  382.047116  2277.863984
36  130702.239410  3626.320072  449.824592  2530.841327
37  116481.873365  4702.782468  472.453725  2597.917632
38   79033.834921  4495.428177  425.359102  2595.857880
39  102257.454089  4800.068321  521.310729  2967.518390
##################### NA #

**Kontrol ve Test Grubunun Verilerini Tek Bir Dataframe'de Birleştirme**

Birleştirme işlemi yapmadan önce kontrol ve test gruplarının değişken isimlerini düzenleyelim.

In [8]:
#Kontrol grubu için
df1.columns = ["Control_" + col for col in df1.columns]
#Test grubu için
df2.columns = ["Test_" + col for col in df2.columns]

In [9]:
df1.head()

,Control_Impression,Control_Click,Control_Purchase,Control_Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [10]:
df2.head()

,Test_Impression,Test_Click,Test_Purchase,Test_Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


In [11]:
#İki dataframe'i tek bir dataframe'de birleştirme

df= pd.concat([df1,df2], axis=1)
df.head()

,Control_Impression,Control_Click,Control_Purchase,Control_Earning,Test_Impression,Test_Click,Test_Purchase,Test_Earning
0,82529.459271,6090.077317,665.211255,2311.277143,120103.503796,3216.547958,702.160346,1939.611243
1,98050.451926,3382.861786,315.084895,1742.806855,134775.943363,3635.082422,834.054286,2929.405820
2,82696.023549,4167.965750,458.083738,1797.827447,107806.620788,3057.143560,422.934258,2526.244877
3,109914.400398,4910.882240,487.090773,1696.229178,116445.275526,4650.473911,429.033535,2281.428574
4,108457.762630,5987.655811,441.034050,1543.720179,145082.516838,5201.387724,749.860442,2781.697521


# **A/B Testinin Yapılması**

> **Hipotezlerin Tanımlanması**

H0: M1 = M2 (Eski teklif türü uygulandığındaki satın alma ortalamaları ile
yeni teklif türü uygulandığındaki satın alma ortalamaları arasında istatistiki
olarak anlamlı fark yoktur.)

H1: M1!=M2 (Eski teklif türü uygulandığındaki satın alma ortalamaları ile
yeni teklif türü uygulandığındaki satın alma ortalamaları arasında istatistiki
olarak anlamlı fark vardır.)


Hipotez testi yapılmadan önce varsayım kontrolleri yapılır. Bu varsayımlar normallik ve varyans homojenliğidir. p value değerleri değerlendirilir.

> **Varsayım Kontrolleri**



In [12]:
#NORMALLİK VARSAYIMI-KONTROL GRUBU:
#H0: Normal dağılım varsayımı sağlanmaktadır.
#H1: Normal dağılım varsayımı sağlanmamaktadır.

test_stat, pvalue= shapiro(df["Control_Purchase"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))


Test Stat = 0.9773, p-value = 0.5891


**Yorum:** H0 red edilemez. 0.05'ten büyüktür. Control_Purchase normal dağılıyor.

In [13]:
#NORMALLİK VARSAYIMI-TEST GRUBU:
#H0: Normal dağılım varsayımı sağlanmaktadır.
#H1: Normal dağılım varsayımı sağlanmamaktadır.

test_stat, pvalue= shapiro(df["Test_Purchase"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


**Yorum:** H0 red edilemez. 0.05'ten büyüktür. Test_Purchase normal dağılıyor.

In [14]:
#VARYANS HOMOJENLİĞİ
#H0: Varyanslar homojendir.
#H1: Varyanslar homojen değildir.

test_stat, pvalue=levene(df["Control_Purchase"],df["Test_Purchase"])
print("Test Stat = %.4f, p-value= %.4f" % (test_stat, pvalue))

Test Stat = 2.6393, p-value= 0.1083


**Yorum:** H0 red edilemez. p-value değeri 0.05'ten büyüktür. Varyanslar homojen dağılmaktadır.


Varsayımlar sağlandığı için paremetrik test uygulanır. Uygulanacak olan test **Bağımsız İki Örneklem t testidir**. p value değerleri değerlendirilir.

Daha öncesinde bu test için hipotezlerimizi yazmıştık ve varsayımlarımızı kontrol etmiştik. Şimdi testimizi yapalım.

In [15]:
test_stat, pvalue = ttest_ind(df["Control_Purchase"], df["Test_Purchase"], equal_var=True)
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


**Yorum:** H0 red edilemez. p value değeri 0.05'ten büyüktür. Kontrol ve test grubunun satın alma ortalamaları arasında istatistiki olarak anlamlı bir fark yoktur.